In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch_geometric

In [3]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cu121.html


In [4]:
cd /content/drive/MyDrive/Academic/Topics/AI/Machine\ Learning\ Dr.\ Montazeri/Project/ml_mda

/content/drive/MyDrive/Academic/Topics/AI/Machine Learning Dr. Montazeri/Project/ml_mda


# Requirement

In [5]:
import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(stream=sys.stdout)
    ],
    force=True
)

In [6]:
from src.models import MDMDAClassifier, MDMDAClassifierFactory
from src.optimization import MDMDAClassifierTester, MDMDAClassifierTrainer
from src.config import MatrixDecomposerConfig, MDMDAClassifierOptimizerConfig
from src.data import MicrobeDiseaseAssociationData, MicrobeDiseaseAssociationTrainTestSpliter
from src.utils import train_test_sampler
from src.features import get_associations, get_entities
from base import cross_validation

2024-02-10 21:03:51,504 [INFO] NumExpr defaulting to 2 threads.


# Classification

## Data

In [7]:
associations = get_associations()

train_indices, test_indices = train_test_sampler(associations.shape[0], 0.7)

data = MicrobeDiseaseAssociationData(associations)

train_data = MicrobeDiseaseAssociationData(associations.iloc[train_indices])
test_data = MicrobeDiseaseAssociationData(associations.iloc[test_indices])

## Classifier

In [8]:
microbe_ids = get_entities().loc[get_entities()['type'] == 'Microbe']['id'].tolist()
disease_ids = get_entities().loc[get_entities()['type'] == 'Disease']['id'].tolist()

In [9]:
classifier_config = MatrixDecomposerConfig()
classifier_config.model_name = "NMF MDA Classifier"
classifier_config.microbe_ids = microbe_ids
classifier_config.disease_ids = disease_ids
classifier_config.n_components = 10
classifier_config.random_state = 1
classifier_config.decomposer = 'NMF'

In [10]:
mda_classifier = MDMDAClassifier(classifier_config)

2024-02-10 21:03:52,189 [INFO] Initializing MCMDAClassifier
2024-02-10 21:03:52,191 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 21:03:52,193 [INFO] Initializing MFFeatureExtractor with model : None and decomposer : NMF


## Optimizer

In [11]:
optimizer_config = MDMDAClassifierOptimizerConfig()
optimizer_config.conv_threshold = 5
optimizer_config.exp_name = "Optimizer for NMF MDA Classifier"
optimizer_config.threshold = 0.5

## Train Test Approach

### Train

In [12]:
train_result = MDMDAClassifierTrainer().train(mda_classifier, train_data, optimizer_config)

2024-02-10 21:03:52,214 [INFO] Call Training with Optimizer for NMF MDA Classifier
2024-02-10 21:03:52,226 [INFO] Calling build with associations :      disease  microbe  increased
728    44112    37477          0
262    33293    13641          1
127    12403    20754          1
804    63129    20627          0
661     1667    46605          0
..       ...      ...        ...
329      654    15670          1
35     43621    27509          1
846    10506     9788          0
228    12403    28933          1
43     50863    65017          1

[628 rows x 3 columns]
2024-02-10 21:03:52,312 [INFO] interaction matrix with shape (5179, 5645) has built
2024-02-10 21:03:52,607 [INFO] mask matrix with shape (5179, 5645) has built. This matrix shows not non elements.
2024-02-10 21:03:54,355 [INFO] interaction has been imputed to delete nans


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:04:54,169 [INFO] interation 1 mse : 30.506063740307727


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:05:51,652 [INFO] interation 2 mse : 21.43196538647058


...

2024-02-10 21:12:16,919 [INFO] interation 9 mse : 5.86358891402908


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:13:12,127 [INFO] interation 10 mse : 5.207539417544243


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:14:04,502 [INFO] interation 11 mse : 4.639842029828027
2024-02-10 21:14:04,507 [INFO] training finished


### Test

In [13]:
test_result = MDMDAClassifierTester().test(model=mda_classifier,
                                            data=test_data,
                                            config=optimizer_config)

2024-02-10 21:14:04,528 [INFO] y_test has been built : [0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1] ...
2024-02-10 21:14:04,725 [INFO] y_predict has been built : [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.7816286454109487, 0.0, 0.9852962209881905, 0.0, 0.0, 0.0, 1.0, 1.0, 0.2055831622000405, 1.0, 0.9888979533412917, 0.0, 0.0, 1.0] ...
2024-02-10 21:14:04,749 [INFO] Test Result : {'AUC': 0.972936512306591, 'ACC': 0.9444444444444444, 'F1 Score': 0.9442233270441118, 'AUPR': 0, 'Loss': 0}


In [14]:
test_result.get_result()

{'AUC': 0.972936512306591,
 'ACC': 0.9444444444444444,
 'F1 Score': 0.9442233270441118,
 'AUPR': 0,
 'Loss': 0}

## Cross Validation

In [15]:
trainer = MDMDAClassifierTrainer()
tester = MDMDAClassifierTester()
factory = MDMDAClassifierFactory(classifier_config)
spliter = MicrobeDiseaseAssociationTrainTestSpliter(data.associations)
cross_validation(k=5, data_size=data.associations.shape[0], train_test_spliter=spliter, model_factory=factory, trainer=trainer, tester=tester, config=optimizer_config)

2024-02-10 21:14:04,774 [INFO] Initializing MCMDAClassifierFactory
2024-02-10 21:14:04,776 [INFO] Initializing MicrobeDiseaseAssociationTrainTestSpliter
2024-02-10 21:14:04,779 [INFO] Start 5-fold Cross Validation with config : Optimizer for NMF MDA Classifier
2024-02-10 21:14:04,782 [INFO] ---- Fold 1 ----
2024-02-10 21:14:04,785 [INFO] Initializing MCMDAClassifier
2024-02-10 21:14:04,787 [INFO] Initializing MatrixFeatureExtractor
2024-02-10 21:14:04,789 [INFO] Initializing MFFeatureExtractor with model : None and decomposer : NMF
2024-02-10 21:14:04,790 [INFO] Call Training with Optimizer for NMF MDA Classifier
2024-02-10 21:14:04,796 [INFO] Calling build with associations :      disease  microbe  increased
0      50863    33211          1
1      43621    40832          1
3      13213    53186          1
4      33293    14909          1
5      33293    35937          1
..       ...      ...        ...
892    22068    20153          0
893    64642    53920          0
894    25026    6

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:15:00,608 [INFO] interation 1 mse : 32.45964921551163


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 21:15:55,361 [INFO] interation 2 mse : 26.089845769794994


...

2024-02-10 22:00:48,801 [INFO] interation 5 mse : 11.881331386148332


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 22:01:41,953 [INFO] interation 6 mse : 10.144086648136136


/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


2024-02-10 22:02:36,535 [INFO] interation 7 mse : 14.595135068663268
2024-02-10 22:02:36,537 [INFO] training finished
2024-02-10 22:02:36,545 [INFO] y_test has been built : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] ...
2024-02-10 22:02:36,613 [INFO] y_predict has been built : [0.9305616405263825, 0.9070033734148907, 0.8799434550567753, 0.8884326722791112, 1.0715298963550886, 0.624905375506367, 0.8287310788988537, 0.9338074186785231, 0.8677082889586158, 1.1286292945228849, 0.3130211813355816, 0.7754029936175756, 0.5152637359136676, 0.5368573607835606, 1.0514093920472576, 1.1147287330769915, 0.8119815566216587, 0.8796177914640178, 0.15849595271474295, 0.8756697983646963] ...
2024-02-10 22:02:36,626 [INFO] Test Result : {'AUC': 0.8916223404255319, 'ACC': 0.8461538461538461, 'F1 Score': 0.8461352657004831, 'AUPR': 0, 'Loss': 0}
2024-02-10 22:02:36,628 [INFO] Result of fold 5 : {'AUC': 0.8916223404255319, 'ACC': 0.8461538461538461, 'F1 Score': 0.8461352657004831, 'AUPR': 